# 1) Cross validation

## Initial calculations

First we import libraries and read in the data.

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut

auto = pd.read_csv('https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Auto.csv', na_values='?').dropna()

Now we split the dataset, half for training, half for testing.

In [2]:
train = auto.sample(frac=0.5, replace=False, random_state=1)
test = auto[~auto.isin(train)].dropna(how = 'all')

train_x = train['horsepower'].values.reshape(-1, 1)
train_y = train['mpg']

test_x = test['horsepower'].values.reshape(-1, 1)
test_y = test['mpg']

First, fit a linear model to the training data.

In [3]:
lm = skl_lm.LinearRegression()
linear_model = lm.fit(train_x, train_y)

Evaluate the linear model against the test data and calculate Mean Squared Error.

In [4]:
linear_pred = linear_model.predict(test_x)
linear_mse = mean_squared_error(test_y, linear_pred)
linear_mse

23.361902892587224

Next, fit a quadratic model to the training data.

In [5]:
pm2 = PolynomialFeatures(degree=2)
train_x_pm2 = pm2.fit_transform(train_x)
test_x_pm2 = pm2.fit_transform(test_x)
quadratic_model = lm.fit(train_x_pm2, train_y)

Evaluate the quadratic model against the test data and calculate Mean Squared Error.

In [6]:
quadratic_pred = quadratic_model.predict(test_x_pm2)
quadratic_mse = mean_squared_error(test_y, quadratic_pred)
quadratic_mse

20.252690858347492

Next, fit a cubic model to the training data.

In [7]:
pm3 = PolynomialFeatures(degree=3)
train_x_pm3 = pm3.fit_transform(train_x)
test_x_pm3 = pm3.fit_transform(test_x)
cubic_model = lm.fit(train_x_pm3, train_y)

Evaluate the cubic model against the test data and calculate Mean Squared Error.

In [8]:
cubic_pred = cubic_model.predict(test_x_pm3)
cubic_mse = mean_squared_error(test_y, cubic_pred)
cubic_mse

20.325609366315604

Next, fit a degree-4 model to the training data.

In [9]:
pm4 = PolynomialFeatures(degree=4)
train_x_pm4 = pm4.fit_transform(train_x)
test_x_pm4 = pm4.fit_transform(test_x)
d4_model = lm.fit(train_x_pm4, train_y)

Evaluate the degree-4 model against the test data and calculate Mean Squared Error.

In [10]:
d4_pred = d4_model.predict(test_x_pm4)
d4_mse = mean_squared_error(test_y, d4_pred)
d4_mse

20.343887109415615

As in the R example, the quadratic model's MSE is the lowest. This might change using a different random seed, however.

## LOOCV calculations


First use LOOCV with a linear model.

In [11]:
# fit linear model
model = lm.fit(train_x, train_y)
loo = LeaveOneOut()
loo_x = auto['horsepower'].values.reshape(-1,1)
loo_y = auto['mpg'].values.reshape(-1,1)
loo.get_n_splits(loo_x)

# using as many folds as we have rows amounts to simple LOOCV
cv = KFold(n_splits=len(auto), random_state=None, shuffle=False)

# calculate MSE
scores = cross_val_score(model, loo_x, loo_y, scoring="neg_mean_squared_error", cv=cv, n_jobs=1)
print(f"MSE for linear model & LOOCV: {str(np.mean(np.abs(scores)))}")

MSE for linear model & LOOCV: 24.231513517929226


Then use LOOCV with polynomial models of increasing degree.

In [12]:
for i in range(1, 6):
    # fit polynomial model of degree i
    poly = PolynomialFeatures(degree=i)
    px = poly.fit_transform(loo_x)
    pm = lm.fit(px, loo_y)
    
    # calculate MSE
    scores = cross_val_score(model, px, loo_y, scoring="neg_mean_squared_error", cv=cv, n_jobs=1)
    print(f"MSE for degree-{i} model & LOOCV: {str(np.mean(np.abs(scores)))}")

MSE for degree-1 model & LOOCV: 24.231513517929226
MSE for degree-2 model & LOOCV: 19.24821312448939
MSE for degree-3 model & LOOCV: 19.334984064114092
MSE for degree-4 model & LOOCV: 19.424430309411886
MSE for degree-5 model & LOOCV: 19.033211842978396


## K-folds CV calculations

Use K-folds CV with polynomial models of increasing degree.

In [13]:
# instead of using as many folds as there are rows, now we K=10
crossvalidation = KFold(n_splits=10, shuffle=False)

for i in range(1, 11):
    # fit polynomial model of degree i
    poly = PolynomialFeatures(degree=i)
    px = poly.fit_transform(loo_x)
    pm = lm.fit(px, loo_y)
    
    # calculate MSE
    scores = cross_val_score(model, px, loo_y, scoring="neg_mean_squared_error", cv=cv, n_jobs=1)
    print(f"MSE for degree-{i} model & K-folds CV: {str(np.mean(np.abs(scores)))}")

MSE for degree-1 model & K-folds CV: 24.231513517929226
MSE for degree-2 model & K-folds CV: 19.24821312448939
MSE for degree-3 model & K-folds CV: 19.334984064114092
MSE for degree-4 model & K-folds CV: 19.424430309411886
MSE for degree-5 model & K-folds CV: 19.033211842978396
MSE for degree-6 model & K-folds CV: 18.973012737758705
MSE for degree-7 model & K-folds CV: 19.126150445808673
MSE for degree-8 model & K-folds CV: 19.22423029373206
MSE for degree-9 model & K-folds CV: 19.133856501117357
MSE for degree-10 model & K-folds CV: 18.945837436861932


References:
- https://www.science.smith.edu/~jcrouser/SDS293/labs/lab7-py.html
- https://stats.stackexchange.com/questions/280665/variance-of-k-fold-cross-validation-estimates-as-fk-what-is-the-role-of
- https://stats.stackexchange.com/questions/61783/bias-and-variance-in-leave-one-out-vs-k-fold-cross-validation